#code aimed at generating a content based recommmender system
#using the Xwines dataset

In [2]:
##standard imports
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix as csr
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from matplotlib import pyplot as plt
import cv2 as cv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import scipy.sparse as sp
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [3]:
##importing the data
user_reviews = pd.read_csv("C:\\Users\\Dell\\reommenders\\RecommenderSystems\\XWines_Slim_150K_ratings.csv", low_memory=False, encoding="utf-8", memory_map=True)
wine_data= pd.read_csv("C:\\Users\\Dell\\reommenders\\RecommenderSystems\\XWines_Slim_1k_wines.csv", low_memory=False, encoding="utf-8", memory_map=True)

#define scalar and encoder
encoder = OneHotEncoder()
scaler = MinMaxScaler()

In [4]:
# Preprocessing for 'Grapes', 'Harmonize', and 'Vintages'
wine_data['Grapes'] = wine_data['Grapes'].apply(lambda x: ' '.join(eval(x)))  # Convert list to space-separated string
wine_data['Harmonize'] = wine_data['Harmonize'].apply(lambda x: ' '.join(eval(x)))




# Function to handle mixed vintages: list or integer
def convert_to_list(vintages):
    if isinstance(vintages, int):
        # If the vintage is a single integer, wrap it in a list
        return [str(vintages)]
    elif isinstance(vintages, str):
        # If it's a string, evaluate it as a list (since strings in CSV could be saved as "['1990', '1995']")
        try:
            return list(map(str, eval(vintages)))  # Convert all elements to strings
        except:
            return [str(vintages)]  # In case it's a string number
    else:
        # In case it's an unexpected format, treat as string
        return [str(vintages)]

# Apply the conversion to the 'Vintages' column
wine_data['Vintages'] = wine_data['Vintages'].apply(convert_to_list)

# Convert the list into a space-separated string
wine_data['Vintages'] = wine_data['Vintages'].apply(lambda x: ' '.join(x))


In [5]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import numpy as np

# TF-IDF for 'Grapes', 'Harmonize', and 'Vintages'
tfidf = TfidfVectorizer()

print(wine_data['Grapes'])
wine_data_tfidf_grapes = tfidf.fit_transform(wine_data['Grapes'])
wine_data_tfidf_harmonize = tfidf.fit_transform(wine_data['Harmonize'])
wine_data_tfidf_vintages = tfidf.fit_transform(wine_data['Vintages'])

# One-hot encoding for 'Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName'
one_hot = OneHotEncoder()

categorical_features = ['Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName']
wine_categorical_onehot = one_hot.fit_transform(wine_data[categorical_features])

# Scale the numerical feature 'ABV'
scaler = StandardScaler()
wine_abv_scaled = scaler.fit_transform(wine_data[['ABV']])



0                  Muscat/Moscato
1                      Ancellotta
2              Cabernet Sauvignon
3       Cabernet Sauvignon Merlot
4           Muscat/Moscato Bianco
                  ...            
1002                Petite Arvine
1003                  St. Laurent
1004                   Chardonnay
1005             Grüner Veltliner
1006                 Syrah/Shiraz
Name: Grapes, Length: 1007, dtype: object


In [6]:
# Define the weights for different features
weights ={'grapes': 2.6351178844772396, 'harmonize': 1.553630179215512, 'vintages': 1.760447782530082, 'type': 2.5859874914949206, 'body': 1.2628975201148853, 'acidity': 0.9828619145406399, 'country': 0.6956261879697497, 'region': 0.6957519257816862, 'winery': 0.7668374998938243, 'abv': 2.11521395128059}

# Apply weights to categorical features (by columns)
wine_type_onehot = wine_categorical_onehot[:, :one_hot.categories_[0].size] * weights['type']  # 'Type' weight
wine_body_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size:one_hot.categories_[0].size + one_hot.categories_[1].size] * weights['body']
wine_acidity_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size] * weights['acidity']
wine_country_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size] * weights['country']
wine_region_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size + one_hot.categories_[4].size] * weights['region']
wine_winery_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size + one_hot.categories_[4].size:] * weights['winery']

# Scale the numerical feature 'ABV'
scaler = StandardScaler()
wine_abv_scaled = scaler.fit_transform(wine_data[['ABV']]) * weights['abv']  # Apply weight for 'ABV'
wine_data_tfidf_grapes = wine_data_tfidf_grapes * weights['grapes']  # Apply weight for 'Grapes'
wine_data_tfidf_harmonize = wine_data_tfidf_harmonize * weights['harmonize']  # Apply weight for 'Harmonize'
wine_data_tfidf_vintages = wine_data_tfidf_vintages * weights['vintages']  # Apply weight for 'Vintages'


In [7]:
wine_features = sp.hstack([
    wine_data_tfidf_grapes, 
    wine_data_tfidf_harmonize, 
    wine_data_tfidf_vintages,
    wine_type_onehot, 
    wine_body_onehot, 
    wine_acidity_onehot,
    wine_country_onehot, 
    wine_region_onehot, 
    wine_winery_onehot, 
    wine_abv_scaled
])

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity between all wines
wine_similarity = cosine_similarity(wine_features)

# Store this in a DataFrame for easier access
wine_similarity_df = pd.DataFrame(wine_similarity, index=wine_data['WineID'], columns=wine_data['WineID'])

print(wine_similarity_df.head()) 

WineID    100001    100002    100003    100005    100007    100008    100010  \
WineID                                                                         
100001  1.000000  0.257048  0.207921  0.274106  0.885468  0.115507  0.047448   
100002  0.257048  1.000000  0.423994  0.457324  0.251593  0.402714  0.323315   
100003  0.207921  0.423994  1.000000  0.761152  0.217980  0.891347  0.498972   
100005  0.274106  0.457324  0.761152  1.000000  0.258979  0.733599  0.419366   
100007  0.885468  0.251593  0.217980  0.258979  1.000000  0.105448  0.015149   

WineID    100012    100013    100014  ...    198580    198885    199204  \
WineID                                ...                                 
100001  0.400310 -0.324444 -0.323762  ...  0.180255  0.157360  0.232856   
100002  0.133552 -0.072407 -0.077954  ...  0.156189  0.139666  0.090151   
100003  0.163455 -0.034760 -0.032770  ...  0.119030  0.103430  0.141385   
100005  0.211784 -0.143106 -0.144681  ...  0.083877  0.157074  0

In [9]:
def recommend_wines(user_id, user_reviews, wine_similarity_df, top_n=5,returnall=False):
    # Get the wines the user has rated
    user_wines = user_reviews[user_reviews['UserID'] == user_id]
    
    # Filter for wines the user rated 4 or 5
    high_rated_wines = user_wines[user_wines['Rating'] >= 4]['WineID']
    
    # Get similarity scores for each of these wines
    similar_wines = pd.Series(dtype=float)
    
    for wine_id in high_rated_wines:
        similar_wines = similar_wines.append(wine_similarity_df[wine_id])
    
    # Sort the wines by similarity score
    similar_wines = similar_wines.groupby(similar_wines.index).mean().sort_values(ascending=False)
    
    # Exclude wines the user has already rated
    similar_wines = similar_wines[~similar_wines.index.isin(user_wines['WineID'])]
    
    if returnall:
        return similar_wines.index.tolist()
    return similar_wines.head(top_n).index.tolist()

# Example recommendation for user with ID 1
recommended_wines = recommend_wines(1356810, user_reviews, wine_similarity_df)
print(recommended_wines)

[111448, 115984, 116266, 116995, 113321]


In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score
from random import random

def evaluate_recommender(user_reviews, wine_similarity_df, top_n=5,threshold=4):
    all_precision_scores = []
    all_recall_scores = []
    all_f1_scores = []

    # Loop through each user in the dataset
    unique=user_reviews['UserID'].unique()
    TP=1
    FP=1
    FN=1
    for user_id in unique:
        # Get all the wines the user has rated 4 or 5
        user_ratings = user_reviews[user_reviews['UserID'] == user_id]
        
        if len(user_ratings) <30:
            continue
        # we want to evaluate for new users
        # so we want to simulate users with few ratings

        # select 10 random ratings
        sparse = user_ratings.sample(10)
        # remove the selected ratings from the user_ratings
        remaining = user_ratings.drop(sparse.index)

        # now we want to evaluate the recommender on the sparse ratings
        # if it recommends wines with a rating of 4 or 5, it's a hit
        recommended_wines = recommend_wines(user_id, sparse, wine_similarity_df, top_n) 
        
        # find wines that the user has rated 
        reviewed_recommended = remaining[remaining['WineID'].isin(recommended_wines)]
        for index, row in reviewed_recommended.iterrows():
            if row['Rating'] >= threshold:
                TP+=1
            else:
                FP+=1
        ## now we want the amount of false negatives
        ## this is the amount of wines that the user rated 4 or 5     

    print("TP:"+str(TP))
    print("FP:"+str(FP))
    print("FN:"+str(FN))
    precision=TP/(TP+FP)
    recall=TP/(TP+FN)
    f1=2*(precision*recall)/(precision+recall)
    # Return the mean precision, recall, and F1-score across all users
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

print(evaluate_recommender(user_reviews,wine_similarity_df,15,threshold=3.5))

TP:469
FP:50
FN:1
{'precision': 0.9036608863198459, 'recall': 0.997872340425532, 'f1': 0.9484327603640041}


block combines all into 1

In [11]:
def evaluate_weights(weights):
    
    weights = {
        'grapes': weights[0],
        'harmonize': weights[1],
        'vintages': weights[2],
        'type': weights[3],
        'body': weights[4],
        'acidity': weights[5],
        'country': weights[6],     # A bit more important
        'region': weights[7],
        'winery': weights[8],      # Slightly less important
        'abv': weights[9]          # Least important
    }

    tfidf = TfidfVectorizer()

    wine_data_tfidf_grapes = tfidf.fit_transform(wine_data['Grapes'])
    wine_data_tfidf_harmonize = tfidf.fit_transform(wine_data['Harmonize'])
    wine_data_tfidf_vintages = tfidf.fit_transform(wine_data['Vintages'])

    # One-hot encoding for 'Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName'
    one_hot = OneHotEncoder()

    categorical_features = ['Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName']
    wine_categorical_onehot = one_hot.fit_transform(wine_data[categorical_features])

    # Scale the numerical feature 'ABV'
    scaler = StandardScaler()
    wine_abv_scaled = scaler.fit_transform(wine_data[['ABV']])
    # Apply weights to categorical features (by columns)
    wine_type_onehot = wine_categorical_onehot[:, :one_hot.categories_[0].size] * weights['type']  # 'Type' weight
    wine_body_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size:one_hot.categories_[0].size + one_hot.categories_[1].size] * weights['body']
    wine_acidity_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size] * weights['acidity']
    wine_country_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size] * weights['country']
    wine_region_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size + one_hot.categories_[4].size] * weights['region']
    wine_winery_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size + one_hot.categories_[4].size:] * weights['winery']

    # Scale the numerical feature 'ABV'
    scaler = StandardScaler()
    wine_abv_scaled = scaler.fit_transform(wine_data[['ABV']]) * weights['abv']  # Apply weight for 'ABV'


    wine_data_tfidf_grapes = wine_data_tfidf_grapes * weights['grapes']  # Apply weight for 'Grapes'
    wine_data_tfidf_harmonize = wine_data_tfidf_harmonize * weights['harmonize']  # Apply weight for 'Harmonize'
    wine_data_tfidf_vintages = wine_data_tfidf_vintages * weights['vintages']  # Apply weight for 'Vintages'
    wine_features = sp.hstack([
        wine_data_tfidf_grapes, 
        wine_data_tfidf_harmonize, 
        wine_data_tfidf_vintages,
        wine_type_onehot, 
        wine_body_onehot, 
        wine_acidity_onehot,
        wine_country_onehot, 
        wine_region_onehot, 
        wine_winery_onehot, 
        wine_abv_scaled
    ])
    # Compute cosine similarity between all wines
    wine_similarity = cosine_similarity(wine_features)

    # Store this in a DataFrame for easier access
    wine_similarity_df = pd.DataFrame(wine_similarity, index=wine_data['WineID'], columns=wine_data['WineID'])

    values=evaluate_recommender(user_reviews,wine_similarity_df)
    return values['precision']


def minimise(weights):
    return -evaluate_weights(weights)




Next part is tweaking the weigths

In [12]:
from pyswarm import pso
import numpy as np
lower_bounds = [0.1] * 10  # Minimum weight is 0.1
upper_bounds = [3.0] * 10  # Maximum weight is 3.0


initial_weights = [2.0, 1.5, 0.8, 1.0, 1.0, 1.0, 1.2, 1.0, 0.9, 0.5]
# Run PSO optimization
# best_weights, best_score = pso(minimise, lower_bounds, upper_bounds, swarmsize=20, maxiter=10)

# # Print the optimal weights found
# optimal_weights = {
#     'grapes': best_weights[0],
#     'harmonize': best_weights[1],
#     'vintages': best_weights[2],
#     'type': best_weights[3],
#     'body': best_weights[4],
#     'acidity': best_weights[5],
#     'country': best_weights[6],
#     'region': best_weights[7],
#     'winery': best_weights[8],
#     'abv': best_weights[9]
# }

# print(f"Optimal Weights: {optimal_weights}")
# print(f"Best Evaluation Score: {best_score}")






In [13]:
##now we extend the model by making it a hybrid model with a collaborative filtering model
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import mean_average_precision_at_k

# Load the CSV file
# Make sure the file path is correct

# Display the first few rows of the DataFrame
df=user_reviews

# Select the relevant columns (UserID, WineID, Rating)
df_filtered = df[['UserID', 'WineID', 'Rating']]

# Create a user-item interaction matrix (rows = UserID, columns = WineID, values = Rating)
user_item_matrix = csr_matrix((df_filtered['Rating'], (df_filtered['UserID'], df_filtered['WineID'])))

# Initialize the ALS model and fit it using the user-item matrix
model = AlternatingLeastSquares(factors=100, regularization=0.01, iterations=50)
model.fit(user_item_matrix)



c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [01:44<00:00,  2.09s/it]


In [14]:
#now predict the ratings for the user
user_id = 1356810
# Get the user's ratings
user_ratings = df_filtered[df_filtered['UserID'] == user_id]

# Predict ratings for all wines
user_wine_scores = model.recommend(user_id, user_item_matrix[user_id], N=10)
print(user_wine_scores)


(array([113288, 113321, 155375, 115365, 111479, 113600, 111448, 113913,
       114606, 101660]), array([0.62537664, 0.48129907, 0.46493974, 0.41493446, 0.39993355,
       0.39101207, 0.37293565, 0.33004454, 0.32436085, 0.32202393],
      dtype=float32))


In [21]:
#lets make a class for the models
#lets make a class for the modelss
class contentBased:
    
    def __init__(self,user_reviews,wine1_data):
        self.user_reviews=user_reviews.copy()
        wine_data=wine1_data.copy()
        # Preprocessing for 'Grapes', 'Harmonize', and 'Vintages'
        print(wine_data['Grapes'])
        wine_data['Grapes'] = wine_data['Grapes'].apply(lambda x: ' '.join(eval(x)))  # Convert list to space-separated string
        wine_data['Harmonize'] = wine_data['Harmonize'].apply(lambda x: ' '.join(eval(x)))




        # Function to handle mixed vintages: list or integer
        def convert_to_list(vintages):
            if isinstance(vintages, int):
                # If the vintage is a single integer, wrap it in a list
                return [str(vintages)]
            elif isinstance(vintages, str):
                # If it's a string, evaluate it as a list (since strings in CSV could be saved as "['1990', '1995']")
                try:
                    return list(map(str, eval(vintages)))  # Convert all elements to strings
                except:
                    return [str(vintages)]  # In case it's a string number
            else:
                # In case it's an unexpected format, treat as string
                return [str(vintages)]

        # Apply the conversion to the 'Vintages' column
        wine_data['Vintages'] = wine_data['Vintages'].apply(convert_to_list)

        # Convert the list into a space-separated string
        wine_data['Vintages'] = wine_data['Vintages'].apply(lambda x: ' '.join(x))

      
        
        tfidf = TfidfVectorizer()
       
        wine_data_tfidf_grapes = tfidf.fit_transform(wine_data['Grapes'])
        wine_data_tfidf_harmonize = tfidf.fit_transform(wine_data['Harmonize'])
        wine_data_tfidf_vintages = tfidf.fit_transform(wine_data['Vintages'])

        # One-hot encoding for 'Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName'
        one_hot = OneHotEncoder()

        categorical_features = ['Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName']
        wine_categorical_onehot = one_hot.fit_transform(wine_data[categorical_features])

        # Scale the numerical feature 'ABV'
        scaler = StandardScaler()
        wine_abv_scaled = scaler.fit_transform(wine_data[['ABV']])

        # Define the weights for different features
        weights ={'Grapes' : 2.6351178844772396, 'Harmonize' : 1.553630179215512, 'Vintages' : 1.760447782530082, 'Type' : 2.5859874914949206, 'Body' : 1.2628975201148853, 'Acidity' : 0.9828619145406399, 'Country' : 0.6956261879697497, 'Region' : 0.6957519257816862, 'Winery' : 0.7668374998938243, 'ABV' : 2.11521395128059}
        # Apply weights to categorical features (by columns)
        wine_type_onehot = wine_categorical_onehot[:, :one_hot.categories_[0].size] * weights['Type']  # 'Type' weight
        wine_body_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size:one_hot.categories_[0].size + one_hot.categories_[1].size] * weights['Body']
        wine_acidity_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size] * weights['Acidity']
        wine_country_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size] * weights['Country']
        wine_region_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size:one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size + one_hot.categories_[4].size] * weights['Region']
        wine_winery_onehot = wine_categorical_onehot[:, one_hot.categories_[0].size + one_hot.categories_[1].size + one_hot.categories_[2].size + one_hot.categories_[3].size + one_hot.categories_[4].size:] * weights['Winery']

        # Scale the numerical feature 'ABV'
        scaler = StandardScaler()
        wine_abv_scaled = scaler.fit_transform(wine_data[['ABV']]) * weights['ABV']  # Apply weight for 'ABV'
        wine_data_tfidf_grapes = wine_data_tfidf_grapes * weights['Grapes']  # Apply weight for 'Grapes'
        wine_data_tfidf_harmonize = wine_data_tfidf_harmonize * weights['Harmonize']  # Apply weight for 'Harmonize'
        wine_data_tfidf_vintages = wine_data_tfidf_vintages * weights['Vintages']  # Apply weight for 'Vintages'
        
        wine_features = sp.hstack([
            wine_data_tfidf_grapes, 
            wine_data_tfidf_harmonize, 
            wine_data_tfidf_vintages,
            wine_type_onehot, 
            wine_body_onehot, 
            wine_acidity_onehot,
            wine_country_onehot, 
            wine_region_onehot, 
            wine_winery_onehot, 
            wine_abv_scaled
        ])
                # Compute cosine similarity between all wines
        wine_similarity = cosine_similarity(wine_features)

        # Store this in a DataFrame for easier access
        wine_similarity_df = pd.DataFrame(wine_similarity, index=wine_data['WineID'], columns=wine_data['WineID'])

        self.wine_similarity_df=wine_similarity_df

    def recommend_wines(self,user_id, top_n=5,returnall=False):
        # Get the wines the user has rated
        user_wines = self.user_reviews[self.user_reviews['UserID'] == user_id]
        # Filter for wines the user rated 4 or 5
        high_rated_wines = user_wines[user_wines['Rating'] >= 4]['WineID']
        # Get similarity scores for each of these wines
        similar_wines = pd.Series(dtype=float)
        for wine_id in high_rated_wines:
            similar_wines = similar_wines.append(self.wine_similarity_df[wine_id])
        # Sort the wines by similarity score
        similar_wines = similar_wines.groupby(similar_wines.index).mean().sort_values(ascending=False)
        # Exclude wines the user has already rated
        similar_wines = similar_wines[~similar_wines.index.isin(user_wines['WineID'])]
        if returnall:
            return similar_wines.index.tolist()
        return similar_wines.head(top_n).index.tolist()
      
class collaborativeFiltering:
    def __init__(self,user_reviews):
        self.user_reviews=user_reviews
        self.df_filtered = self.user_reviews[['UserID', 'WineID', 'Rating']]
        self.user_item_matrix = csr_matrix((self.df_filtered['Rating'], (self.df_filtered['UserID'], self.df_filtered['WineID'])))
        self.model = AlternatingLeastSquares(factors=100, regularization=0.01, iterations=50)
        self.model.fit(self.user_item_matrix)
    def recommend_wines(self,user_id, top_n=5,returnall=False):
        # Get the user's ratings
        user_ratings = self.df_filtered[self.df_filtered['UserID'] == user_id]
        # Predict ratings for all wines
        user_wine_scores = self.model.recommend(user_id, self.user_item_matrix[user_id], N=10)
        if returnall:
            return user_wine_scores
        return user_wine_scores.head(top_n).index.tolist()

class hybridModel:
    def __init__(self,user_reviews,wine_data,ratio):
        self.user_reviews=user_reviews
        self.wine_data=wine_data
        self.contentBasedModel=contentBased(self.user_reviews,self.wine_data)
        self.collaborativeFilteringModel=collaborativeFiltering(self.user_reviews)
        self.ratio=ratio	
    def recommend_wines(self,user_id, top_n=5,returnall=False):
        contentBasedWines=self.contentBasedModel.recommend_wines(user_id, top_n,returnall)
        collaborativeFilteringWines=self.collaborativeFilteringModel.recommend_wines(user_id, top_n,returnall)
        allWines=ratio*contentBasedWines+(1-ratio)*collaborativeFilteringWines
        return allWines

    

                                                                  

In [22]:

##reinport the data
user_reviews = pd.read_csv("C:\\Users\\Dell\\reommenders\\RecommenderSystems\\XWines_Slim_150K_ratings.csv", low_memory=False, encoding="utf-8", memory_map=True)
wine_data= pd.read_csv("C:\\Users\\Dell\\reommenders\\RecommenderSystems\\XWines_Slim_1k_wines.csv", low_memory=False, encoding="utf-8", memory_map=True)


In [29]:


#filter out red wines
wine_data=wine_data[wine_data['Type']=='Red']
#filter out reviews for red wines
user_reviews=user_reviews[user_reviews['WineID'].isin(wine_data['WineID'])]
##instantiate the content based model
contentBasedModel=contentBased(user_reviews,wine_data)
# contentBasedModel.preprocess()
contentBasedModel.recommend_wines(1173759, top_n=5,returnall=False)


1                             ['Ancellotta']
2                     ['Cabernet Sauvignon']
3           ['Cabernet Sauvignon', 'Merlot']
5                     ['Cabernet Sauvignon']
6       ['Touriga Nacional', 'Syrah/Shiraz']
                        ...                 
983                               ['Merlot']
985                           ['Pinot Noir']
989                          ['St. Laurent']
995                           ['Pinot Noir']
1003                         ['St. Laurent']
Name: Grapes, Length: 506, dtype: object


[111431, 111433, 111417, 111932, 113193]